# Breakout v1 - Analytics

### Import Library

In [29]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [30]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [31]:
data = dataset.copy()

In [32]:
data = data[data.index > '2024-01-01 01:00:00']

In [33]:
TREND_UP = 'up'
TREND_DOWN = 'down'
def prepare_data(htd):
    htd["current"] = htd.index + pd.DateOffset(minutes=5)
    htd["ema_f1"] = ta.ema(htd["Close"], length=9)
    htd["ema_l1"] = ta.ema(htd["Close"], length=26)
    htd["short_trend"] = htd.apply(lambda r: TREND_UP if r['ema_f1'] > r['ema_l1'] else TREND_DOWN, axis=1)
    htd["ema_f2"] = ta.ema(htd["Close"], length=16)
    htd["ema_l2"] = ta.ema(htd["Close"], length=250)
    htd["long_trend"] = htd.apply(lambda r: TREND_UP if r['ema_f2'] > r['ema_l2'] else TREND_DOWN, axis=1)
    htd.drop(columns=['ema_f1', 'ema_l1', 'ema_f2', 'ema_l2'], inplace=True)
    htd["min_oc"] = htd.apply(lambda r: r['Open'] if r['Open'] < r['Close'] else r['Close'], axis=1)
    htd["max_oc"] = htd.apply(lambda r: r['Open'] if r['Open'] > r['Close'] else r['Close'], axis=1)
    htd.dropna(inplace=True)
    htd = htd[100 * htd.index.hour + htd.index.minute != 1130]
    return htd

In [34]:
data = prepare_data(data)

In [35]:
def cal_signal(row):
    signal = ''
    if row['long_trend']==TREND_UP:
        if row['Close'] - row['Open'] > 2.49:
            signal = 'signal'
        elif row['max_oc'] - row['min_oc'] > 2.49:
            signal = 'signal'

In [42]:
def cal_signal(data):
    for i, row in data.iterrows():
        signal = ''
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = max_price = min_price = row['Close']
        data_from_start_day = data[(data.index < current_time) & (data.index > current_date + ' 08:59:00')]
        if abs(row['Close'] - row['Open']) > 2.49:
            if row['long_trend']==TREND_UP or row['long_trend']==TREND_DOWN:
                signal = 'found'        
        data.at[i, 'signal'] = signal
    return data

In [43]:
data = cal_signal(data)

In [44]:
data.tail(50)

,Open,High,Low,Close,Volume,current,short_trend,long_trend,min_oc,max_oc,signal
Date,,,,,,,,,,,
2024-09-18 09:00:00,1305.0,1305.7,1304.0,1304.0,4270,2024-09-18 09:05:00,up,up,1304.0,1305.0,
2024-09-18 09:05:00,1304.5,1305.4,1304.1,1304.8,1994,2024-09-18 09:10:00,up,up,1304.5,1304.8,
2024-09-18 09:10:00,1305.0,1305.2,1304.6,1305.2,895,2024-09-18 09:15:00,up,up,1305.0,1305.2,
2024-09-18 09:15:00,1305.2,1305.4,1304.2,1304.3,2945,2024-09-18 09:20:00,up,up,1304.3,1305.2,
2024-09-18 09:20:00,1304.2,1304.2,1303.2,1303.5,2921,2024-09-18 09:25:00,up,up,1303.5,1304.2,
2024-09-18 09:25:00,1303.5,1304.6,1303.5,1304.2,1447,2024-09-18 09:30:00,up,up,1303.5,1304.2,
2024-09-18 09:30:00,1304.2,1307.5,1303.6,1307.1,4492,2024-09-18 09:35:00,up,up,1304.2,1307.1,signal
2024-09-18 09:35:00,1307.4,1307.4,1306.1,1306.4,3595,2024-09-18 09:40:00,up,up,1306.4,1307.4,
2024-09-18 09:40:00,1306.4,1307.5,1306.3,1306.6,2583,2024-09-18 09:45:00,up,up,1306.4,1306.6,
